In [ ]:
import http.server
import json
import mysql.connector
from urllib.parse import urlparse

# Database connection
db_config = {
    'user': 'root',
    'password': 'RootRoot',
    'host': 'localhost',
    'port': 3306,
    'database': 'contactlist2'
}

class ContactList:
    def __init__(self):
        self.conn = mysql.connector.connect(**db_config)
        self.cursor = self.conn.cursor(dictionary=True) # holds the rs

    def create_contact(self, fullName, phone, email):
        query = "INSERT INTO contacts (fullName, phone, email) VALUES (%s, %s, %s)"
        self.cursor.execute(query, (fullName, phone, email))
        self.conn.commit()
        return {"id": self.cursor.lastrowid, "fullName": fullName, "phone": phone, "email": email}

    def read_contacts(self):
        self.cursor.execute("SELECT * FROM contacts")
        return self.cursor.fetchall()

    def get_contact_by_id(self, contact_id):
        query = "SELECT * FROM contacts WHERE id = %s"
        self.cursor.execute(query, (contact_id,))
        return self.cursor.fetchone()

    def update_contact(self, contact_id, new_phone=None, new_email=None):
        query = "UPDATE contacts SET phone = %s, email = %s WHERE id = %s"
        self.cursor.execute(query, (new_phone, new_email, contact_id))
        self.conn.commit()
        return self.cursor.rowcount > 0

    def delete_contact(self, contact_id):
        query = "DELETE FROM contacts WHERE id = %s"
        self.cursor.execute(query, (contact_id,))
        self.conn.commit()
        return self.cursor.rowcount > 0

contact_list = ContactList()

class RequestHandler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        parsed_path = urlparse(self.path)
        path_parts = parsed_path.path.split('/')
        if len(path_parts) == 2 and path_parts[1] == "contacts":
            self.send_response(200)
            self.send_header("Content-type", "application/json")
            self.end_headers()
            self.wfile.write(json.dumps(contact_list.read_contacts()).encode())
        elif len(path_parts) == 3 and path_parts[1] == "contacts":
            contact_id = path_parts[2]
            contact = contact_list.get_contact_by_id(contact_id)
            if contact:
                self.send_response(200)
                self.send_header("Content-type", "application/json")
                self.end_headers()
                self.wfile.write(json.dumps(contact).encode())
            else:
                self.send_response(404)
                self.end_headers()
        else:
            self.send_response(404)
            self.end_headers()

    def do_POST(self):
        if self.path == "/contacts":
            content_length = int(self.headers['Content-Length'])
            post_data = self.rfile.read(content_length)
            data = json.loads(post_data)
            new_contact = contact_list.create_contact(data['fullName'], data['phone'], data['email'])
            self.send_response(201)
            self.send_header("Content-type", "application/json")
            self.end_headers()
            self.wfile.write(json.dumps(new_contact).encode())
        else:
            self.send_response(404)
            self.end_headers()

    def do_PUT(self):
        parsed_path = urlparse(self.path)
        path_parts = parsed_path.path.split('/')
        if len(path_parts) == 3 and path_parts[1] == "contacts":
            contact_id = path_parts[2]
            content_length = int(self.headers['Content-Length'])
            put_data = self.rfile.read(content_length)
            data = json.loads(put_data)
            updated = contact_list.update_contact(contact_id, data.get('phone'), data.get('email'))
            if updated:
                self.send_response(200)
                self.send_header("Content-type", "application/json")
                self.end_headers()
                self.wfile.write(json.dumps({"message": "Contact updated successfully"}).encode())
            else:
                self.send_response(404)
                self.end_headers()
        else:
            self.send_response(404)
            self.end_headers()

    def do_DELETE(self):
        parsed_path = urlparse(self.path)
        path_parts = parsed_path.path.split('/')
        if len(path_parts) == 3 and path_parts[1] == "contacts":
            contact_id = path_parts[2]
            if contact_list.delete_contact(contact_id):
                self.send_response(200)
                self.end_headers()
            else:
                self.send_response(404)
                self.end_headers()
        else:
            self.send_response(404)
            self.end_headers()

def run(server_class=http.server.HTTPServer, handler_class=RequestHandler, port=8080):
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)
    print(f'Starting server on port {port}...')
    httpd.serve_forever()

if __name__ == "__main__":
    run()
